- we have cleaned and explored the data
- now i'll be using cleaned dataset
- First spliting the data and as 60% training, 20% validation and 20% for testing and then
- I'll be using 5 fold cross-validation on 5 algorithms
- Logistic Regression
- Support vector machine(SVM)
- Multilayer preceptron(MLP)
- Random Forest
- Boosting

In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
df = pd.read_csv('cleaned.csv')

In [5]:
df.head()

,Survived,Pclass,Sex,Age,Fare,Fam_cnt,cabin_ind
0,0,3,1,22.0,7.2500,1,0
1,1,1,0,38.0,71.2833,1,1
2,1,3,0,26.0,7.9250,0,0
3,1,1,0,35.0,53.1000,1,1
4,0,3,1,35.0,8.0500,0,0


# Spliting the data for train, test and validation

In [8]:
features = df.drop('Survived',axis=1)
label = df['Survived']

In [10]:
features.head()

,Pclass,Sex,Age,Fare,Fam_cnt,cabin_ind
0,3,1,22.0,7.2500,1,0
1,1,0,38.0,71.2833,1,1
2,3,0,26.0,7.9250,0,0
3,1,0,35.0,53.1000,1,1
4,3,1,35.0,8.0500,0,0


In [11]:
label.head()

0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

In [14]:
x_train,x_test,y_train,y_test = train_test_split(features,label,test_size=0.4,random_state=42)
x_val,x_test,y_val,y_test = train_test_split(x_test,y_test,test_size=0.5,random_state=42)

In [15]:
x_train.to_csv('train_features.csv',index=False)
y_train.to_csv('train_label.csv',index=False)
x_test.to_csv('test_features.csv',index=False)
y_test.to_csv('test_label.csv',index=False)
x_val.to_csv('val_features.csv',index=False)
y_val.to_csv('val_label.csv',index=False)

In [28]:
for dataset in [y_train, y_val, y_test]:
    print(round(len(dataset) / len(label),1))

0.6
0.2
0.2


In [23]:
for dataset in [y_train, y_val, y_test]:
    print(len(dataset))

534
178
179
